# Практическое задание

## Урок 5. Рекуррентные нейронные сети

1. Попробуйте изменить параметры нейронной сети, работающей с датасетом imdb, либо нейронной сети, работающей c airline-passengers так, чтобы улучшить её точность. Приложите анализ

## Импорт библиотек и модулей

In [1]:
# математика
import numpy
import math

import pandas as pd

# визуализация 
import matplotlib.pyplot as plt


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score

# для создания комбинаций параметров
from itertools import product

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Формирование тренировочной и тестовой подвыборок

In [2]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

numpy.random.seed(7)

dataframe = pd.read_csv('airline-passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

## Настройки параметров нейронной сети 

In [3]:
batch_size = ['1', '2']
epochs = ['1', '2', '3', '4', '5']
eta = ['0.01', '0.1']

# список для сохранения результатов 
list_metrics_pivot = []
list_cols = ['batch_size', 'epochs', 'Train Score (RMSE)', 'Test Score (RMSE)']

In [8]:
def nn_airline_passengers(trainX, trainY, testX, testY, batch_size, epochs, eta):
    model = Sequential()
    model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='mean_squared_error', optimizer=Adam(eta))

    for i in range(100):
        history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False)
        
        model.reset_states()
        
    trainPredict = model.predict(trainX, batch_size=batch_size)
    model.reset_states()
    testPredict = model.predict(testX, batch_size=batch_size)

    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    
#     accuracy = round(float(history.history['accuracy'][0])*100, 2)
#     accuracy = round(float(accuracy_score(trainY, trainPredict)*100), 2)

    trainScore = round(math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0])), 2)
#     print('Train Score: %.2f RMSE' % (trainScore))
    testScore = round(math.sqrt(mean_squared_error(testY[0], testPredict[:,0])), 2)
#     print('Test Score: %.2f RMSE' % (testScore))

    list_metrics_pivot.append([batch_size, epochs, trainScore, testScore])
    
#     trainPredictPlot = numpy.empty_like(dataset)
#     trainPredictPlot[:, :] = numpy.nan
#     trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

#     testPredictPlot = numpy.empty_like(dataset)
#     testPredictPlot[:, :] = numpy.nan
#     testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

# plt.plot(scaler.inverse_transform(dataset))
# plt.plot(trainPredictPlot)
# plt.plot(testPredictPlot)
# plt.show()


In [9]:
list_combs_params = list(product(batch_size, epochs, eta))    

In [10]:
%%time

for i in range(len(list_combs_params)):
    nn_airline_passengers(trainX=trainX, 
                          trainY=trainY, 
                          testX=testX, 
                          testY=testY, 
                          batch_size=int(list_combs_params[i][0]), 
                          epochs=int(list_combs_params[i][1]), 
                          eta=float(list_combs_params[i][2]))
    
# CPU times: total: 12min 17s
# Wall time: 8min 53s

23/23 [==============================] - 0s 1ms/step
CPU times: total: 15min 39s
Wall time: 10min 52s


In [11]:
# сводный датасет 
data_total = pd.DataFrame(list_metrics_pivot, columns=list_cols)
data_total.sort_values(by=list_cols[2], ascending=True).head(3)

,batch_size,epochs,Train Score (RMSE),Test Score (RMSE)
16,2,4,15.06,76.99
14,2,3,17.73,55.72
12,2,2,18.98,98.05


Вывод: Улучшение качества работы нейронной сети достигается путем увеличения параметра epochs и cнижения batch_size

---

2. Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. Можно использовать текст другого произведения

In [12]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN, LSTM, GRU
from keras.models import Sequential

In [13]:
# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 64, 64
NUM_ITERATIONS = 25 
NUM_EPOCHS_PER_ITERATION = 2
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(
    LSTM(HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['accuracy'])


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
Epoch 1/2
2476/2476 [==============================] - 10s 3ms/step - loss: 2.4829 - accuracy: 0.3040
Epoch 2/2
2476/2476 [==============================] - 8s 3ms/step - loss: 2.1132 - accuracy: 0.3913
Генерация из посева: plates, an
plates, and and and the said the said the said the said the said the said the said the said the said the said ==================================================
Итерация #: 1
Epoch 1/2
2476/2476 [==============================] - 9s 4ms/step - loss: 1.9717 - accuracy: 0.4280
Epoch 2/2
2476/2476 [==============================] - 10s 4ms/step - loss: 1.8674 - accuracy: 0.4572
Генерация из посева: i should l
i should litt of the was a dore the coure the coure the coure the coure the coure the coure the coure the cour==================================================
Итерация #: 2
Epoch 1/2
2476/2476 [==============================] - 9s 4ms/step - loss: 1.7886 - accuracy: 0.4795
Epoch 2/2
2476/2476 [==============================] - 13s 5ms/st

deed! said the gryphon was the mock turtle and the mock turtle and the mock turtle and the mock turtle and the==================================================
Итерация #: 20
Epoch 1/2
2476/2476 [==============================] - 12s 5ms/step - loss: 1.2846 - accuracy: 0.6186
Epoch 2/2
2476/2476 [==============================] - 12s 5ms/step - loss: 1.2818 - accuracy: 0.6187
Генерация из посева: nly too mu
nly too must be a little should the works of the same the same the same the same the same the same the same th==================================================
Итерация #: 21
Epoch 1/2
2476/2476 [==============================] - 9s 4ms/step - loss: 1.2785 - accuracy: 0.6200
Epoch 2/2
2476/2476 [==============================] - 9s 4ms/step - loss: 1.2757 - accuracy: 0.6198
Генерация из посева: d in reply
d in reply, and the mock turtle said to the dormouse she took the words of the took the words of the took the ==================================================
Итерация #: 22
Ep

Вывод: Для улучшения работы нейронной сети увеличено число количество эпох за итерацию и количество итераций